# Heating Mesh Tally

***This constructs a reactor geomerty and material from 3 volumes, the breeder blanket, the PF magnets and a central column shield. It simulates this using a mesh tally producing the nuclear hetaing in the form of mesh.***

This makes a 3D geometry and material for PF coil

In [ ]:
import paramak


pf_coil = paramak.RotateStraightShape(
    points=[
        (500, 500),
        (550, 500),
        (550, 550),
        (500, 550)],
    stp_filename = 'pf_coil.stp',
    material_tag = 'pf_coil_material')

pf_coil.solid

This makes a 3D geometry and material for the centre column

In [ ]:
center_column = paramak.RotateMixedShape(
    points=[
        (50, 600, 'straight'),
        (150, 600, 'spline'),
        (100, 0, 'spline'),
        (150, -600, 'straight'),
        (50, -600, 'straight')],
    stp_filename = 'center_column.stp',
    material_tag = 'center_column_material')

center_column.solid

This makes a 3D geometry and material for breeder blanket

In [ ]:
blanket = paramak.RotateSplineShape(
    points=[
        (600, 0),
        (600, -20),
        (500, -300),
        (400, -300),
        (400, 0),
        (400, 300),
        (500, 300),
        (600, 20)],
    rotation_angle=40,
    azimuth_placement_angle=[0, 45, 90, 135, 180, 225, 270, 315],
    stp_filename = 'blanket.stp',
    material_tag = 'blanket_material')
blanket.solid

In [ ]:
my_reactor = paramak.Reactor([blanket, pf_coil,center_column])

In [ ]:
my_reactor.export_stp()

In [ ]:
!ls

In [ ]:
from neutronics_material_maker import Material

mat1 = Material(material_name='Li4SiO4',
                material_tag='blanket_material')

mat2 = Material(material_name='copper',
                material_tag='pf_coil_material')

mat3 = Material(material_name='WC',
                material_tag='center_column_material')

In [ ]:
from parametric_plasma_source import PlasmaSource, SOURCE_SAMPLING_PATH
import openmc

my_plasma = PlasmaSource(
    ion_density_origin=1.09e20,
    ion_density_peaking_factor=1,
    ion_density_pedestal=1.09e20,
    ion_density_separatrix=3e19,
    ion_temperature_origin=45.9,
    ion_temperature_peaking_factor=8.06,
    ion_temperature_pedestal=6.09,
    ion_temperature_separatrix=0.1,
    elongation=1.3,
    triangularity=0.5,
    major_radius=200/100,
    minor_radius=50/100,
    pedestal_radius=(0.8 * 100)/100,
    plasma_id=1,
    shafranov_shift=0.44789,
    ion_temperature_beta=6
)

source = openmc.Source()
source.library = SOURCE_SAMPLING_PATH
source.parameters = str(my_plasma)

In [ ]:
neutronics_model = paramak.NeutronicsModel(
    geometry=my_reactor,
    cell_tallies=['heating'],
    source=source,
    simulation_batches=10,
    simulation_particles_per_batch=10,
    materials={
        'blanket_material': mat1,
        'pf_coil_material': mat2,
        'center_column_material': mat3,
    }
)

neutronics_model.simulate(method='pymoab')

In [ ]:
neutronics_model.results